# 4. Tile output images 
**Requirements**: 
Familiarity with python, dependencies, and managing environments.  

## Folder structure
Output images may be too big for python to process as a whole image.  
The code below tiles them. A folder named "tiles" will appear in each image folder.  

- **export** 
    - `name_image_1`
      - `fluid.tif` 
      - `tissue.tif`
      - `tiles`
    - `name_image_2`
    - `name_image_n`

## Run the code 
First, make sure all necessary dependencies are installed 
Edit the python code as follows:  
**```parent_folder```**: enter the path to the ```export``` folder  
**```gapname```**: enter the name of the mask images containing the gaps  
**```tissuename```**: enter the name of the mask images containing the tissue  

This code may take a while to run. Check the ```export``` folder for ```tiles``` 

In [2]:
import numpy as np 
from PIL import Image
import os
import glob
from skimage import io
Image.MAX_IMAGE_PIXELS = 5000000000


parent_folder = './edema_tutorial_example/qupath_project/export/' # Path to export folder generated by export_edema_experimental.groovy. 
gapname = 'fluid.tif'
tissuename = 'tissue.tif'

# This folder contains subfolders with the images that we want to tile.

def list_folders(parent_folder):
    # Use glob to find all directories in the parent folder
    folders = [f for f in glob.glob(os.path.join(parent_folder, '*')) if os.path.isdir(f)]
    return folders


def tile_image(image, output_dir, tile_size, name=''):
    # Load the image

    imagePIL = Image.fromarray((image * 255).astype(np.uint8)) #if image.dtype == np.float else Image.fromarray(image)
    image_width, image_height = imagePIL.size

    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    if tile_size > image_width or tile_size > image_height:
        print(f"Tile size {tile_size} is larger than image dimensions {image_width}x{image_height}. Reducing tile size.")
        tile_size = max(image_width, image_height)
        

    # Loop through and save tiles
    tile_count = 0
    for x in range(0, image_width, tile_size):
        for y in range(0, image_height, tile_size):
            # Define the box to extract the tile
            box = (x, y, x + tile_size, y + tile_size)
            tile = imagePIL.crop(box)
            
            # Save the tile
            tile_path = os.path.join(output_dir, f'{name}tile_{tile_count}.png')
            tile.save(tile_path)
            tile_count += 1

folders = list_folders(parent_folder)


## Load the images
for folder in folders:
    # check if output folder exists
    output = folder + '/tiles'
    if os.path.exists(output):
        print(f"Folder {folder} already processed")
        continue

    if not os.path.exists(folder + "/" + tissuename):
        print(f"Folder {folder} does not contain {tissuename}")
        continue


    imageexo_full = io.imread(folder + '/' + tissuename, as_gray=True)
    image_full = io.imread(folder + '/' + gapname, as_gray=True)

    # # Load the images
    tile_image(image_full, output, 10000)
    tile_image(imageexo_full, output, 10000, name='exo')



for folder in folders:
    # check if output folder exists
    output = folder + '/tiles'
    if os.path.exists(output):
        continue
    elif not os.path.exists(folder + '/' + gapname):
        print(f"Folder {folder} does not contain {gapname}")
        continue

    else: 
        print(f"Folder {folder} needs processing")

print(folders)

Tile size 10000 is larger than image dimensions 2272x2272. Reducing tile size.
Tile size 10000 is larger than image dimensions 2272x2272. Reducing tile size.
Tile size 10000 is larger than image dimensions 2048x2240. Reducing tile size.
Tile size 10000 is larger than image dimensions 2048x2240. Reducing tile size.
['./edema_tutorial_example/qupath_project/export/sick', './edema_tutorial_example/qupath_project/export/healthy']
